In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

# Analyze predictions stored by the API/performance tracker
conn = sqlite3.connect('predictions.db')

df = pd.read_sql("SELECT * FROM predictions ORDER BY predicted_at DESC", conn)
print('Rows in predictions:', len(df))
print(df.head())

# Basic metrics for last 30 days
cutoff = (datetime.now() - timedelta(days=30)).isoformat()
recent = pd.read_sql("SELECT * FROM predictions WHERE predicted_at > ?", conn, params=(cutoff,))
if len(recent):
    acc = (recent['prediction'] == recent['actual_outcome']).mean() if 'actual_outcome' in recent.columns and recent['actual_outcome'].notna().any() else None
    print('Recent rows:', len(recent), 'Accuracy:', acc)
else:
    print('No recent predictions found.')


In [ ]:
# 1) Basic counts and latest timestamps
print('Total rows:', len(df))
if len(df):
    print('First ts:', df['predicted_at'].min())
    print('Last ts:', df['predicted_at'].max())
    print('Unique symbols:', df['symbol'].nunique())
    print(df.head(10))


In [ ]:
# 2) Recent accuracy and win rate
if len(df):
    mask = df['actual_outcome'].notna()
    eval_df = df[mask].copy()
    if len(eval_df):
        acc = (eval_df['prediction'] == eval_df['actual_outcome']).mean()
        win_rate = (eval_df['actual_return'] > 0).mean()
        print('Overall accuracy:', round(float(acc), 4))
        print('Overall win rate:', round(float(win_rate), 4))
    else:
        print('No rows with actual outcomes yet.')


In [ ]:
# 3) Sharpe-like metric on realized returns
import numpy as np

if len(df) and 'actual_return' in df.columns and df['actual_return'].notna().any():
    r = df['actual_return'].dropna()
    sharpe = r.mean() / (r.std() + 1e-9) * np.sqrt(252)
    print('Sharpe (approx):', round(float(sharpe), 3))
else:
    print('No realized returns available for Sharpe calculation.')


In [ ]:
# 4) Per-symbol performance summary
if len(df):
    if 'actual_return' in df.columns:
        by_symbol = df.groupby('symbol')['actual_return'].agg(['mean','std','count']).reset_index()
        by_symbol = by_symbol.sort_values('mean', ascending=False)
        print(by_symbol.head(10))
    else:
        print('actual_return column missing. Run performance tracker updates.')


In [ ]:
# 5) Confusion matrix from realized outcomes
from sklearn.metrics import confusion_matrix

mask = df['actual_outcome'].notna()
if mask.any():
    cm = confusion_matrix(df.loc[mask,'actual_outcome'], df.loc[mask,'prediction'], labels=[-1,0,1])
    print('Confusion matrix:')
    print(cm)
else:
    print('No actual outcomes yet to compute confusion matrix.')


In [ ]:
# 6) Daily prediction volume and signal counts
if len(df):
    counts = df.groupby(['predicted_at']).size().rename('count').reset_index()
    print(counts.head(10))
    sig_counts = df.groupby(['predicted_at','prediction']).size().unstack(fill_value=0)
    print(sig_counts.head(10))


In [ ]:
# 7) Confidence calibration (if confidence logged)
if 'confidence' in df.columns:
    conf_desc = df['confidence'].describe()
    print('Confidence stats:', conf_desc.to_dict())
    by_pred = df.groupby('prediction')['confidence'].mean().round(3)
    print('Avg confidence by predicted class:', by_pred.to_dict())
else:
    print('No confidence column found.')


In [ ]:
# 8) Latency/throughput placeholders
# If API logged latency, you could analyze here. For now, placeholders:
api_metrics = {
    'p50_ms': None,
    'p90_ms': None,
    'requests_per_min': None,
}
print('API metrics:', api_metrics)


In [ ]:
# 9) Export a daily KPI table
if len(df) and 'actual_return' in df.columns:
    daily_kpi = df.groupby(df['predicted_at'].str[:10]).agg(
        n=('symbol','count'),
        win_rate=('actual_return', lambda s: (s > 0).mean() if len(s) else None),
        avg_return=('actual_return','mean')
    ).reset_index().rename(columns={'predicted_at':'date'})
    print(daily_kpi.head(10))
else:
    print('Not enough data to compute daily KPI.')


In [ ]:
# 10) Top/bottom symbols by realized return (recent)
if len(df) and 'actual_return' in df.columns:
    recent_mask = df['predicted_at'] >= cutoff
    recent_df = df.loc[recent_mask].copy()
    agg = recent_df.groupby('symbol')['actual_return'].mean().reset_index()
    print('Top 10 symbols (avg return):')
    print(agg.sort_values('actual_return', ascending=False).head(10))
    print('Bottom 10 symbols (avg return):')
    print(agg.sort_values('actual_return', ascending=True).head(10))
else:
    print('No recent realized returns to rank symbols.')


In [ ]:
# 11) Export report CSVs
out_dir = 'data/predictions'
os.makedirs(out_dir, exist_ok=True)

df.to_csv(os.path.join(out_dir, 'predictions_dump.csv'), index=False)
print('Wrote', os.path.join(out_dir, 'predictions_dump.csv'))

# Save daily KPI if computed above
try:
    daily_kpi.to_csv(os.path.join(out_dir, 'daily_kpi.csv'), index=False)
    print('Wrote daily_kpi.csv')
except Exception as e:
    print('daily_kpi not available:', e)


In [ ]:
# 12) Summary and next steps
print('Analysis complete.')
print('Suggested next steps:')
print('- Improve label quality and update_actual_outcomes scheduling.')
print('- Calibrate thresholds and explore cost-sensitive metrics.')
print('- Add drift and data quality dashboards if needed.')
